# Multi-document LangGraph with Redis Retrieval Agent using Ollama and React agents
[Retrieval Agents](https://python.langchain.com/docs/use_cases/question_answering/conversational_retrieval_agents) are useful when we want to make decisions about whether to retrieve from an index. Here we construct the same single index of 10Ks with metadata that we tried in notebook 04.

To implement a Redis retrieval agent, we simply need to give an LLM access to a Redis retriever tool.

We can incorporate this into [LangGraph](https://python.langchain.com/docs/langgraph).

In [9]:
import json
import os
import warnings
warnings.filterwarnings("ignore")
dir_path = os.getcwd()
parent_directory = os.path.dirname(dir_path)
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["ROOT_DIR"] = parent_directory
print(dir_path)
print(parent_directory)

/Users/rouzbeh.farahmand/PycharmProjects/commit/financial-vss/multi_doc_RAG
/Users/rouzbeh.farahmand/PycharmProjects/commit/financial-vss


In [10]:
!pip3 install -r $ROOT_DIR/requirements.txt

In [11]:
# Replace values below with your own if using Redis Cloud instance
REDIS_HOST = os.getenv("REDIS_HOST", "localhost")
REDIS_PORT = os.getenv("REDIS_PORT", "6379")
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")
os.environ["OPENAI_API_KEY"] = 'sk-proj-LjOKui5AFXvDfGcssZECT3BlbkFJeIIVTcAEqxFtNZztsQg1'
os.environ["NLTK_DATA"] = "/Users/rouzbeh.farahmand/nltk_data"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = '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'
# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://{REDIS_HOST}:{REDIS_PORT}"
os.environ["REDIS_URL"] = REDIS_URL

### SentenceTransformerEmbeddings Models Cache folder
We are using `SentenceTransformerEmbeddings` in this demo and here we specify the cache folder. If you already downloaded the models in a local file system, set this folder here, otherwise the library tries to download the models in this folder if not available locally.

In particular, these models will be downloaded if not present in the cache folder:

models/models--sentence-transformers--all-MiniLM-L6-v2

models/models--sentence-transformers--all-mpnet-base-v2


In [12]:
#setting the local downloaded sentence transformer models f
os.environ["TRANSFORMERS_CACHE"] = f"{parent_directory}/models"

In [13]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

embeddings = SentenceTransformerEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                           cache_folder=os.getenv("TRANSFORMERS_CACHE", f"{parent_directory}/models"))

### Build your Redis index 
Skip this section if you have already built your index in previous notebook.


In [14]:
from redisvl.index import SearchIndex
from redisvl.schema import IndexSchema
from redis import Redis
index_name = 'langchain'
prefix = 'chunk'
schema = IndexSchema.from_yaml('sec_index.yaml')
client = Redis.from_url(REDIS_URL)
# create an index from schema and the client
index = SearchIndex(schema, client)
index.create(overwrite=True, drop=True)

14:01:27 redisvl.index.index INFO   Index already exists, overwriting.


In [15]:
# Skip if you have already done populated your index.
from ingestion import get_sec_data
from ingestion import redis_bulk_upload

sec_data = get_sec_data()

 ✅ Loaded doc info for  112 tickers...


In [16]:
redis_bulk_upload(sec_data, index, embeddings, tickers=['AAPL'])

✅ Loaded 108 10K chunks for ticker=AAPL from AAPL-2021-10K.pdf
✅ Loaded 94 10K chunks for ticker=AAPL from AAPL-2023-10K.pdf
✅ Loaded 103 10K chunks for ticker=AAPL from AAPL-2022-10K.pdf
✅ Loaded 27 earning_call chunks for ticker=AAPL from 2018-May-01-AAPL.txt
✅ Loaded 31 earning_call chunks for ticker=AAPL from 2019-Oct-30-AAPL.txt
✅ Loaded 30 earning_call chunks for ticker=AAPL from 2016-Jan-26-AAPL.txt
✅ Loaded 31 earning_call chunks for ticker=AAPL from 2020-Jul-30-AAPL.txt
✅ Loaded 30 earning_call chunks for ticker=AAPL from 2017-Aug-01-AAPL.txt
✅ Loaded 29 earning_call chunks for ticker=AAPL from 2020-Jan-28-AAPL.txt
✅ Loaded 34 earning_call chunks for ticker=AAPL from 2016-Apr-26-AAPL.txt
✅ Loaded 29 earning_call chunks for ticker=AAPL from 2017-Jan-31-AAPL.txt
✅ Loaded 28 earning_call chunks for ticker=AAPL from 2019-Apr-30-AAPL.txt
✅ Loaded 26 earning_call chunks for ticker=AAPL from 2017-Nov-02-AAPL.txt
✅ Loaded 31 earning_call chunks for ticker=AAPL from 2016-Oct-25-AAPL.tx

## Redis as a Retriever


In [17]:
from langchain_community.vectorstores import Redis as LangChainRedis
from utils import create_langchain_schemas_from_redis_schema

index_name = 'langchain'

vec_schema , main_schema = create_langchain_schemas_from_redis_schema('sec_index.yaml')

rds = LangChainRedis.from_existing_index( embedding = embeddings, 
                                          index_name = index_name, 
                                          schema = main_schema)
redis_retriever = rds.as_retriever()


Test if the Redis index is working and returning relevant document.

In [18]:
rds.similarity_search(query="Apple in 2022", k=4, distance_threshold=0.8)

[Document(page_content='The Company’s global operations are subject to complex and changing laws and regulations on subjects, including antitrust; privacy, data security and data localization; consumer protection; advertising, sales, billing and e-commerce; financial services and technology; product liability; intellectual property ownership and infringement; digital platforms; internet, telecommunications, and mobile communications; media, television, film and digital content; availability of third-party software applications and services; labor and employment; anticorruption; import, export and trade; foreign exchange controls and cash repatriation restrictions; anti–money laundering; foreign ownership and investment; tax; and environmental, health and safety, including electronic waste, recycling, and climate change.\n\nApple Inc. | 2022 Form 10-K | 13', metadata={'id': 'chunk:AAPL-2022-10K.pdf-610f6b95-1b3c-4eb7-8b31-9d6fdebf68a8', 'chunk_id': 'AAPL-2022-10K.pdf-610f6b95-1b3c-4eb7-

Now we create a Redis retriever tool.

In [19]:
import operator
from datetime import datetime
from typing import Annotated, TypedDict, Union

from langchain.agents import create_react_agent
from langchain_community.chat_models import ChatOllama
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
from langchain_core.tools import tool
from langgraph.graph import END, StateGraph
from langgraph.prebuilt import ToolExecutor, ToolInvocation

@tool
def get_relevant_docs_from_redis(input: str):
    """
    Get the relevant docs from a redis query.
    """
    redis_response = rds.similarity_search(query=input, k=4, distance_threshold=0.8)
    constructed_context = " "
    for doc in redis_response:
        constructed_context = constructed_context +" \n " + doc.page_content
    if len(redis_response) > 0:
        return str(redis_response[0].page_content).replace("\n", " ")
    return "No relevant docs found in Redis"


tools = [get_relevant_docs_from_redis]

tool_executor = ToolExecutor(tools)


In [20]:
from langchain_core.prompts import PromptTemplate
from typing import Annotated, Sequence, TypedDict

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages


class AgentState(TypedDict):
    # The add_messages function defines how an update should be processed
    # Default is to replace. add_messages says "append"
    input: str
    chat_history: list[BaseMessage]
    agent_outcome: Union[AgentAction, AgentFinish, None]
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]
 
model = ChatOllama(model="llama3")
local_prompt = PromptTemplate(
        template="""Answer the following questions as best you can. You have access to the following    tools:

            {tools}
            
            Use the following format:
            
            Question: the input question you must answer
            Thought: you should always think about what to do
            Action: the action to take, should be one of [{tool_names}]
            Action Input: the input to the action
            Observation: the result of the action is a snippet of financial information related to a company
            ... (this Thought/Action/Action Input/Observation can repeat N times)
            Action Input: Use the observation and try to come up with an answer, if you cannot come up with a number or an answer, only summarize the relevant information that you have and site the source. 
            Thought: I now know the final answer
            Final Answer: the final answer to the original input question or a summary of the relevant information you got from the information you got in the Observation
            
            Begin!
            
            Question: {input}
            Thought:{agent_scratchpad}""",
        input_variables=["tools","tool_names", "input","agent_scratchpad"],
    )


agent_runnable = create_react_agent(model, tools, local_prompt)


In [21]:
def execute_tools(state):
    print("Called `execute_tools`")
    messages = [state["agent_outcome"]]
    last_message = messages[-1]

    tool_name = last_message.tool

    print(f"Calling tool: {tool_name}")

    action = ToolInvocation(
        tool=tool_name,
        tool_input=last_message.tool_input,
    )
    response = tool_executor.invoke(action)
    return {"intermediate_steps": [(state["agent_outcome"], response)]}


def run_agent(state):
    """
    #if you want to better manages intermediate steps
    inputs = state.copy()
    if len(inputs['intermediate_steps']) > 5:
        inputs['intermediate_steps'] = inputs['intermediate_steps'][-5:]
    """
    print(f"RFD-DEBUG:==========")
    print(f"RFD-DEBUG:{str(state)}")
    print(f"RFD-DEBUG:============")
    agent_outcome = agent_runnable.invoke(state)
    print(f"RFD-DEBUG:agent_outcome{str(agent_outcome)}")
    return {"agent_outcome": agent_outcome}


def should_continue(state):
    messages = [state["agent_outcome"]]
    last_message = messages[-1]
    if "Action" not in last_message.log:
        return "end"
    else:
        return "continue"

In [22]:
workflow = StateGraph(AgentState)

workflow.add_node("agent", run_agent)
workflow.add_node("action", execute_tools)


workflow.set_entry_point("agent")

workflow.add_conditional_edges(
    "agent", should_continue, {"continue": "action", "end": END}
)


workflow.add_edge("action", "agent")
app = workflow.compile()

input_text = "Whats the revenue of Apple in 2022?"

In [23]:
inputs = {"input": input_text, "chat_history": []}
results = []
for s in app.stream(inputs):
    result = list(s.values())[0]
    results.append(result)
    print(result)

RFD-DEBUG:==========
RFD-DEBUG:{'input': 'Whats the revenue of Apple in 2022?', 'chat_history': [], 'agent_outcome': None, 'intermediate_steps': []}
RFD-DEBUG:============
RFD-DEBUG:agent_outcometool='get_relevant_docs_from_redis' tool_input='revenue apple 2022' log='Thought: To find the revenue of Apple for 2022, I\'ll start by searching through relevant documents stored in Redis.\n\nAction: get_relevant_docs_from_redis\nAction Input: "revenue apple 2022'
{'agent_outcome': AgentAction(tool='get_relevant_docs_from_redis', tool_input='revenue apple 2022', log='Thought: To find the revenue of Apple for 2022, I\'ll start by searching through relevant documents stored in Redis.\n\nAction: get_relevant_docs_from_redis\nAction Input: "revenue apple 2022')}
Called `execute_tools`
Calling tool: get_relevant_docs_from_redis
{'intermediate_steps': [(AgentAction(tool='get_relevant_docs_from_redis', tool_input='revenue apple 2022', log='Thought: To find the revenue of Apple for 2022, I\'ll start b

OutputParserException: Could not parse LLM output: `Based on the provided documents from Redis and the Apple Inc. Form 10-K for 2022, I will attempt to answer your question about Apple's revenue for 2022.

From the "Segment Information and Geographic Data" note in the Form 10-K (Note 11), we can see that Apple's net sales by disaggregated revenue source was generally consistent across reportable segments in 2022. However, there is no specific figure mentioned for Apple's total revenue in 2022.

Instead, I'll focus on the other relevant information provided in the Form 10-K. In Note 3 – Financial Instruments: Cash, Cash Equivalents and Marketable Securities, we see that as of September 24, 2022, Apple had a total of $183,061 million (Fair Value) in cash, cash equivalents, and marketable securities.

This figure is not directly related to revenue, but it gives an idea of the company's liquidity and financial position. Unfortunately, this information does not provide a direct answer to yo